#### Import Libs

In [289]:
import os
import re

####  File Utils

In [290]:
def read_file(folder, filename):
    filepath = os.path.join(folder, filename)
    file = open(filepath, 'r')
    page = file.read()
    return page

In [400]:
link_regex = '<a href=\"http://(.*?).html'

#### Page Rank

In [439]:
def open_pages(folder):
    filenames = os.listdir(folder)
    page_names = list(map(lambda filename: filename.split('.')[0], filenames))
    pages = []
    for filename in filenames:        
        page = read_file(folder, filename)
        pages.append(page)
    return (pages, page_names)
    

def page_rank_row(page, page_names):
    # n pages
    n = len(page_names)
    
    # search links in the page
    matchs = re.findall(link_regex, page)
    
    # build adjacency
    row = []
    if len(matchs) == 0:
        row = [1/n, 1/n, 1/n, 1/n, 1/n, 1/n]
    else:
        for page_name in pages_names:
            n_matchs = 0
            for match in matchs:
                if page_name == match:
                    n_matchs += 1
            row.append(n_matchs / len(matchs))
    
    return row

def page_rank(pages, page_names):
    matrix = np.zeros(shape=(len(page_names), len(page_names)))
    for index, page in enumerate(pages):
        row = page_rank_row(page, page_names)
        matrix[index] = row
    return matrix


def power_iteration(A, num_iterations: int):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_iterations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

array([0.70710961, 0.70710395])

#### Run

In [445]:
folder = 'paginas'
pages, page_names = open_pages(folder)
matrix = page_rank(pages, page_names)
print(matrix)
# resuelvo el sistema usando power iteration
power_iteration(matrix, 10000)

[[0.         0.         1.         0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         0.         0.66666667 0.33333333]
 [0.         0.         0.         0.5        0.         0.5       ]
 [0.         0.25       0.25       0.         0.5        0.        ]]


array([0.40824829, 0.40824829, 0.40824829, 0.40824829, 0.40824829,
       0.40824829])